# <center>Big Data for Engineers – Exercises</center>
## <center>Spring 2025 – Week 8 – ETH Zurich</center>
## <center>Spark - Solutions </center>

## 0. Preparation for the exercise in Spark

1. Drag this notebook and everything else in the exercise08 folder in the `notebooks` folder of your exam magic box

2. Start docker with ```docker-compose up -d```

3. Launch Jupiter from the docker container

## 1. What is YARN?

Fundamentally, “**Y**et **A**nother **R**esource **N**egotiator”. **YARN**  is a resource scheduler designed to work on existing and new Hadoop clusters. 

YARN supports pluggable schedulers. The task of the scheduler is to share the resources of a large cluster among different tenants (applications) while trying to meet application demands (memory, CPU). A user may have several applications active at a time. 

### 1.1 List at least 3 main shortcomings of MapReduce v1 that are addressed by YARN.

### 1.2 State which of the following statements are true:

1. The ResourceManager has to provide fault tolerance for resources across the cluster 

2. Container allocation/deallocation can take place in a dynamic fashion as the application progresses. 

3. YARN plans to allow applications to only request resources in terms of memory usage and number of CPUs.

4. Communications between the ResourceManager and NodeManagers are heartbeat-based. 

5. The ResourceManager does not have a global view of all usage of cluster resources. Therefore, it tries to make better scheduling decisions based on probabilistic prediction. 

6. ResourceManager has the ability to request resources back from a running application.

### 1.3 Whose responsibility is it? Say which component of YARN is resposible for each of the following tasks.

1. Fault Tolerance of running applications *[ResourceManager | ApplicationMaster | NodeManager ]*
1. Asking for resources needed for an application *[ResourceManager | ApplicationMaster | NodeManager ]*

1. Providing leases to use containers *[ResourceManager | ApplicationMaster | NodeManager]*

1. Tracking status and progress of running applications *[ResourceManager | ApplicationMaster | NodeManager]*

### 1.4 What is the typical configuration for YARN? Choose for the following components how many instances of them there are in a cluster.

```
1. ResourceManager                  a. One per cluster

2. ApplicationMaster                b. One per node

3. NodeManager                      c. Many per cluster, but usually not per node

4. Container                        d. Many per node 
```

## 2. Apache Spark Architecuture

Spark is a fast and general-purpose cluster computing platform. It builds upon the MapReduce model and extends it to efficiently support a broader range of workloads, including interactive queries and stream processing—tasks that traditionally required separate distributed systems.

At a high level, every Spark application consists of a **driver program** that launches various parallel operations on a cluster. The driver program contains the application's main function and defines distributed datasets on the cluster, then applies operations to them.

Driver programs access Spark through a **SparkContext** object, which represents a connection to a computing cluster. There is no need to create a SparkContext; it is created for you automatically when you run the first code cell in the Jupyter.

The driver communicates with a potentially large number of distributed workers called **executors**. The driver runs in its own process and each executor is a separate process. A driver and its executors are together termed a Spark application.

![Image of Account](http://spark.apache.org/docs/latest/img/cluster-overview.png)

### 2.1 Understand Resilient Distributed Datasets (RDDs)

An RDD in Spark is simply an immutable distributed collection of objects. Each RDD is split into multiple partitions, which may be computed on different nodes of the cluster. 

#### What are RDD operations?
RDDs offer two types of operations: **transformations** and **actions**.

* **Transformations** create a new dataset (still RDD) from an existing one. Transformations are lazy, meaning that no transformation is executed until you execute an action.
* **Actions** compute a result (maybe a single value) based on an RDD, and either return it to the driver program or save it to an external storage system (e.g., HDFS). Actions trigger the execution of all requested transformations.




Transformations and actions are different because of the way Spark computes RDDs. Although you can define new RDDs any time, Spark computes them only in a **lazy** fashion, that is, the first time they are used in an action. We are going to talk more about this in a bit.

#### How do I make an RDD?

RDDs can be created from stable storage or by transforming other RDDs. In this exercise, we will run the cells below to create RDDs from local files. Generally it is possible to read data from other resources using the following tokens:

* `file`: Read from the local file system.
* `hdfs`: Read from a Hadoop Distributed File System.
* `s3`  : Read from AWS S3 Storage.
* `wasb`: Read from Azure Blob Storage.

In [ ]:
from pyspark.context import SparkContext
# sc is the Spark Context object 
sc = SparkContext('local', 'test')

In [ ]:
fruits = sc.textFile('./fruits.txt')
yellowThings = sc.textFile('./yellowthings.txt')

### 2.2 RDD transformations
Following are examples of some of the common transformations available. For a detailed list, see [RDD Transformations](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations).

Run some transformations below to understand this better.

**Note1:** the `collect` command is NOT a Transformation, it is an Action used here for the purposes of showing the results! \
**Note2:** If some of the queries are taking too long to complete, try restarting the kernel, and rerunning the cell *above*.

In [ ]:
# map
fruitsReversed = fruits.map(lambda fruit: fruit[::-1])
fruitsReversed.collect()

In [ ]:
# filter
shortFruits = fruits.filter(lambda fruit: len(fruit) <= 5)
shortFruits.collect()

In [ ]:
# flatMap
characters = fruits.flatMap(lambda fruit: list(fruit))
characters.collect()

In [ ]:
# union
fruitsAndYellowThings = fruits.union(yellowThings)
fruitsAndYellowThings.collect()

In [ ]:
# intersection
yellowFruits = fruits.intersection(yellowThings)
yellowFruits.collect()

In [ ]:
# distinct
distinctFruitsAndYellowThings = fruitsAndYellowThings.distinct()
distinctFruitsAndYellowThings.collect()

In [ ]:
# groupByKey
yellowThingsByFirstLetter = yellowThings.map(lambda thing: (thing[0], thing)).groupByKey()
for letter, lst in yellowThingsByFirstLetter.collect():
    print("For letter", letter)
    for obj in lst:
        print(" > ", obj)

In [ ]:
# reduceByKey
numFruitsByLength = fruits.map(lambda fruit: (len(fruit), 1)).reduceByKey(lambda x, y: x + y)
numFruitsByLength.collect()

### 2.3 RDD actions
Following are examples of some of the common actions available. For a detailed list, see [RDD Actions](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions).

Run some transformations below to understand this better. 

In [ ]:
# collect
fruitsArray = fruits.collect()
yellowThingsArray = yellowThings.collect()
print(fruitsArray)
print(yellowThingsArray)

In [ ]:
# count
numFruits = fruits.count()
numFruits

In [ ]:
# take
first3Fruits = fruits.take(3)
first3Fruits

In [ ]:
# reduce
letterSet = fruits.map(lambda fruit: set(fruit)).reduce(lambda x, y: x.union(y))
# Note1: the set() operation converts each string into a set of unique characters.
# Note2: The union used here is a set union, meaning it combines the elements of two sets and removes duplicates automatically.
letterSet

### 2.4 Lazy evaluation
Lazy evaluation means that when we call a transformation on an RDD (for instance, calling `map()`), the operation is not immediately performed. Instead, Spark internally records metadata to indicate that this operation has been requested. Rather than thinking of an RDD as containing specific data, it is best to think of each RDD as
consisting of instructions on how to compute the data that we build up through transformations. Loading data into an RDD is lazily evaluated in the same way transformations are. So, when we call `sc.textFile()`, the data is not loaded until it is necessary. As with transformations, the operation (in this case, reading the data) can
occur multiple times.


Finally, as you derive new RDDs from each other using transformations, Spark keeps track of the set of dependencies between different RDDs, called the lineage graph. For instance, the code bellow corresponds to the following graph:

<img src="https://polybox.ethz.ch/index.php/s/TH3rEYtgRDByKqE/download/Stage5.png" width="400">

**Remember that in order to obtain data from the node on the lineage graph (our RDD of interest) you have to call an action!**

In [ ]:
apples = fruits.filter(lambda x: "apple" in x)
lemons = yellowThings.filter(lambda x: "lemon" in x)
applesAndLemons = apples.union(lemons)
print(applesAndLemons.collect())
# We can also print the lineage graph
print(applesAndLemons.toDebugString().decode("utf-8"))

### 2.5 Persistence (Caching)

Spark's RDDs are by default recomputed each time you run an action on
them. If you would like to reuse an RDD in multiple actions, you can ask Spark to persist it using `RDD.persist()`. After computing it the first time, Spark will store the RDD contents in memory (partitioned across the machines in your cluster), and reuse them in future actions. Persisting RDDs on disk instead of memory is also possible.

If you attempt to cache too much data to fit in memory, Spark will automatically evict old partitions using a Least Recently Used (LRU) cache policy. For the memory-only storage levels, it will recompute these partitions the next time they are accessed,
while for the memory-and-disk ones, it will write them out to disk. In either case, this means that you don't have to worry about your job breaking if you ask Spark to cache too much data. However, caching unnecessary data can lead to eviction of useful data
and more recomputation time. Finally, RDDs come with a method called `unpersist()` that lets you manually remove them from the cache.


To give a simple example to motivate the use of caching, let us consider the following case. Assume we have some a sample of points in our RDD and we want to compute different statistics - mean, minimum and maximum values. However, we also want to first prepare our data by doing some heavy preprocessing. If we do not use caching, we would have to recompute the preprocessing stage for each statistic, while by caching the preprocessed RDD will make us preprocess our data only once.

In [ ]:
fruits.persist()
numFruits = fruits.count()
first3Fruits = fruits.take(3)
fruits.unpersist()
print(numFruits,first3Fruits)

### 2.6 Working with Key/Value Pairs


Spark provides special operations on RDDs containing key/value pairs. These RDDs
are called *pair RDDs*. Pair RDDs are a useful building block in many programs, as
they expose operations that allow you to act on each key in parallel or regroup data
across the network. For example, pair RDDs have a `reduceByKey()` method that can
aggregate data separately for each key, and a `join()` method that can merge two
RDDs together by grouping elements with the same key. Pair RDDs are also still RDDs. 



In [ ]:
rdd = sc.parallelize([("key1", 0), ("key2", 3), ("key1", 8), ("key3", 3), ("key3", 9)])
rdd2 = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
print(rdd2.collect())

### 2.7 Some practice
1. What does the above code do?

2. Consider the following RDD ```sales_rdd``` and write a program to compute the quantity sold for each product

In [ ]:
sales_rdd = sc.parallelize([
    ("productA", 5),
    ("productB", 3),
    ("productA", 2),
    ("productC", 7),
    ("productB", 1),
    ("productC", 4),
    ("productC", 2),
    ("productD", 12),
    ("productA", 1),
    ("productB", 8),
    ("productB", 9)
])

### 2.8 Converting a user program into tasks

A Spark driver is responsible for converting a user program into units of physical execution called tasks. At a high level, all Spark programs follow the same structure: they create RDDs from some input, derive new RDDs from those using transformations, and perform actions to collect or save data. A Spark program implicitly creates a logical **directed acyclic graph (DAG)** of operations.
When the driver runs, it converts this logical graph into a physical execution plan.

Spark performs several optimizations, such as "pipelining" map transformations together to merge them, and converts the execution graph into a set of **stages**.
Each stage, in turn, consists of multiple tasks. The tasks are bundled up and prepared to be sent to the cluster. Tasks are the smallest unit of work in Spark; a typical user program can launch hundreds or thousands of individual tasks.

Each RDD maintains a pointer to one or more parents along with metadata about what
type of relationship they have. For instance, when you call `val b = a.map()` on an
RDD, the RDD `b` keeps a reference to its parent `a`. These pointers allow an RDD to be
traced to all of its ancestors.

The following phases occur during Spark execution:
* User code defines a DAG (directed acyclic graph) of RDDs. Operations on RDDs create new RDDs that refer back to their parents, thereby creating a graph.
* Actions force translation of the DAG to an execution plan. When you call an action on an RDD, it must be computed. This requires computing its parent RDDs as well. 
* Spark's scheduler submits a job to compute all needed RDDs. That job will have one or more stages, which are parallel waves of computation composed of tasks. Each stage will correspond to one or more RDDs in the DAG. A single stage can correspond to multiple RDDs due to pipelining.
* Tasks are scheduled and executed on a cluster
* Stages are processed in order, with individual tasks launching to compute segments of the RDD. Once the final stage is finished in a job, the action is complete.

## 3. The Great Language Game

We will now explore the dataset that is going to be used in the graded exercise of this week. Follow the instructions below to build ```confusion-part.json```.

1. Move to the `notebooks` folder in the terminal
2. Download the data: <br>
   ```wget https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2``` <br>
   __or__ <br>
   ```curl -O https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2```
3. Extract the data: <br>
   ```tar -jxvf confusion-2014-03-02.tbz2```
4. Change directory to ```confusion-2014-03-02```
5. Extract the part of the dataset that we will work with in this exercise: <br>
   ```head -n 3000000 confusion-2014-03-02.json > confusion-part.json```

Next, load the data.

In [ ]:
path = "./confusion-2014-03-02/confusion-part.json"
dataset = sc.textFile(path)

Since the entries are JSON records, you will need to parse them and use their respective object representations. You can use this mapping for all queries. Since some of the queries take a long time to execute on the dataset, you may want to answer these queries on the first `100000` entries. 

**For the rest of the exercise, fill in the results by running the queries on the created subset instead of the entire dataset.**

In [ ]:
import json

entries = dataset.map(json.loads).cache()
print(entries)

And test it. Is it working? You probably have noticed that we are just declaring RDDs without evaluating them. Now let's evaluate it using the 'take' action and look at the json objects. We are goint to look at one of the entries for which the target language is German.

In [ ]:
target_german = entries.filter(lambda e: e["target"] == "German").take(1)
# We use json.dumps() to turn the entry into a nicely formatted string
print(json.dumps(target_german, indent = 4))

Good! Let's get to work. A few last things:
- Take into account that some of the queries might have very large outputs, which Jupyter (or sometimes even Spark) won't be able to handle. It is normal for the queries to take some time, but if the notebook crashes or stops responding, try restarting the kernel. Avoid printing large outputs. You can print the first few entries to confirm the query has worked, as shown in query 1 with ```take(n)```.
- Refer to the [documentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html), as well as the programming guides on actions and transformations linked to above.

And now to the actual queries: *Please make sure that in your queries you *only* use PySpark, and avoid any dataframes (they will covered in next week's exercises)*

### 3.1 How many games have been played by germans (country = "DE")?

### 3.2 Find the number of games such that the target language is German and such that the country is Germany (DE). 

### 3.3 Find the percentage of games where Germans got to hear their native language. Round your answer to an integer. For example: 15, NOT 0.146, 14.6, 14, 14%, 15.1

### 3.4 Find games where Swiss players (country = CH) got to hear one of the official Swiss languages (target is in `official_swiss_languages`). Group the number of games by language and return the language with the highest number of games.

**Hint**: use `countByKey`.

In [ ]:
official_swiss_languages = ["German", "French", "Italian", "Romansh"]

### 3.5 For each country, compute the percentage of games with more than 3 choices (i.e. *len(choices)* > *3*) relative to all games for that country, and display the pairs `(country, percentage)` in the descending order of the percentage. What is the second country in this list? 

**Hint**: use `groupByKey` to aggregate per country and `mapValues` to compute the percentage.

In [ ]:
correct = test_entries.map(lambda e: (e["country"], len(e["choices"]) > 3)).groupByKey()
outcomes = correct.mapValues(lambda e: float(list(e).count(True))/len(e)).sortBy(lambda e: e[1], ascending=False).collect()
print(outcomes[:10])

## 4. Some theory

1. Why is Spark faster than Hadoop MapReduce?

2. Which of the graphs below are DAGs?
    
<img src="https://polybox.ethz.ch/index.php/s/P4GMvUmQDLjFTRK/download" style="width: 700px;">

## 5. True or False
Say if the following statements are *true* or *false*, and explain why.

1. Each RDD is split into multiple partitions, which may be computed on different nodes of the cluster.
1. Transformations construct a new RDD from a previous one and immediately calculate the result
1. Spark's RDDs are by default recomputed each time you run an action on them
1. After computing an RDD, Spark will automatically store its contents in memory and reuse them in future actions.
1. When you derive new RDDs using transformations, Spark keeps track of the set of dependencies between different RDDs.